## Simple Langchain Gen-AI App


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
## Data Ingestion from Website
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.langchain.com/oss/python/langchain/knowledge-base")
loader

2026-01-07 14:13:05.000316: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-07 14:13:05.045301: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-07 14:13:06.215555: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/abhi/.virtualenvs/my_genai_venv/lib/python3.11/site-packages/torch/cuda/__

In [4]:
documents = loader.load()
documents

[Document(metadata={'source': 'https://docs.langchain.com/oss/python/langchain/knowledge-base', 'title': 'Build a semantic search engine with LangChain - Docs by LangChain', 'language': 'en'}, page_content='Build a semantic search engine with LangChain - Docs by LangChainSkip to main contentDocs by LangChain home pageLangChain + LangGraphSearch...⌘KSupportGitHubTry LangSmithTry LangSmithSearch...NavigationLangChainBuild a semantic search engine with LangChainLangChainLangGraphDeep AgentsIntegrationsLearnReferenceContributePythonLearnTutorialsLangChainSemantic searchRAG agentSQL agentVoice agentMulti-agentLangGraphConceptual overviewsComponent architectureMemoryContextGraph APIFunctional APIAdditional resourcesLangChain AcademyCase studiesGet helpOn this pageOverviewConceptsSetupInstallationLangSmith1. Documents and Document LoadersLoading documentsSplitting2. Embeddings3. Vector stores4. RetrieversNext stepsTutorialsLangChainBuild a semantic search engine with LangChainCopy pageCopy pa

In [5]:
## Divide documents into chunks -> Convert to Embeddings -> Store in Vector DB
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
doc_splits = text_splitter.split_documents(documents)


In [6]:
len(doc_splits)

42

In [7]:
from langchain_openai import AzureOpenAIEmbeddings
api_version = "2024-12-01-preview" 

embeddings = AzureOpenAIEmbeddings(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    model = os.getenv("AZURE_OPENAI_EMBEDDINGS_MODEL"),
    api_version = api_version
)

embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x702d8ab976d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x702d8a5e80d0>, model='text-embedding-3-large', dimensions=None, deployment=None, openai_api_version='2024-12-01-preview', openai_api_base=None, openai_api_type='azure', openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=2048, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True, azure_endpoint='https://pfs-2-namit-resource.cognitiveservices.azure.com/', azure_ad_token=None, azure_ad_token_provider=None, azure_ad_async_token_provider=None, valida

In [8]:
from langchain_community.vectorstores import FAISS
vector_db = FAISS.from_documents(doc_splits, embeddings)
vector_db

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context.
    
    <context>
    {context}
    </context>
    
    Question: {input}
    
    Answer:
    """
)

In [10]:
from langchain_openai import AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment = os.getenv("AZURE_OPENAI_LLM_MODEL"),
    api_version = "2025-01-01-preview",
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
)

In [11]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [12]:
document_chain = prompt | llm | output_parser

In [13]:
document_chain.invoke({"input":"How to build a semantic search enginer using LangChain?", 
"context": doc_splits[0]
})

'To build a semantic search engine using **LangChain**, you typically follow these main steps (as outlined in the LangChain documentation):\n\n1. **Load Documents** – Start by gathering your data and loading it into LangChain using *Document Loaders*. These loaders handle different data sources such as local files, web pages, or APIs.\n\n2. **Split Documents** – Break large documents into smaller, more manageable text chunks. This helps improve retrieval accuracy since embeddings work best on moderate-length text segments.\n\n3. **Create Embeddings** – Convert text chunks into numerical vector representations using an *Embedding Model* (for example, OpenAI embeddings or other supported models).\n\n4. **Store in a Vector Store** – Save the embeddings and their associated text in a *Vector Store* (like FAISS, Chroma, or Pinecone). This enables efficient similarity search.\n\n5. **Set Up a Retriever** – Build a retriever interface that queries the vector store to find and return the most 

In [14]:
## Documents to come from retreiver
retreiver = vector_db.as_retriever()

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retreiver | format_docs, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

This is the heart of our pipeline. Let’s unpack it.

**Step 1 — Parallel Stage: `{"context": retriever | format_docs, "input": RunnablePassthrough()}`**

This is a dictionary of runnables. Because it’s a dict, this part of the pipeline becomes a parallel stage.

What each key does

* context:
    * The retriever is invoked with the user’s question.
    * It returns a list of docs.
    * format_docs then runs your format_docs() function on the docs.
    * The result is a string of text containing all relevant document content.

* input:
    * RunnablePassthrough() simply passes the user’s input through unchanged.
    * This is needed because after parallel retrieval, you still need the original user query visible to the next stage.

So after this step, the data shape is: `{
    "context": "some text from docs…",
    "input": "What is X?"
}`

<br>

**Step 2 — Prompt Template: `| prompt`**

Here, prompt is a Runnable that expects a dict. The runnable takes the small dict from the previous step and formats it into a prompt string for the LLM.

In current case we have 2 variables in PromptTemplate so it expects from previous stage `{"context": ..., "input": ...}`

If our prompt had three placeholders, then it would expect: `{"var1": ..., "var2": ..., "var3": ...}`

<br>

**Step 3 — LLM Call: `| llm`**

The output of the prompt runnable (the filled-in prompt text) becomes the input to the AzureChatOpenAI LLM.

This step calls the Azure endpoint with: prompt text, deployed model API key, etc. and produces raw LLM output.

<br>

**Step 4 — Output Parsing: `| StrOutputParser()`**

Finally, we chain a parser runnable that:
* takes raw LLM output
* returns a cleaned up string

This replaces older output cleaning logic.

In [17]:
response = rag_chain.invoke("How to build a semantic search engine using LangChain?")
response

'To build a semantic search engine using **LangChain**, follow these main steps:\n\n1. **Load and Prepare Documents**  \n   - Use LangChain’s **document loaders** (for example, the `pypdf` loader) to import your source data, such as a PDF file.  \n   - Split longer texts into smaller chunks using **text splitters** so that embeddings can be computed efficiently and with meaningful context.\n\n2. **Generate Text Embeddings**  \n   - Choose an **embedding model** to convert text chunks into numerical vectors.  \n   - These vectors capture semantic meaning, enabling similarity-based retrieval.\n\n3. **Store Embeddings in a Vector Store**  \n   - Initialize a **VectorStore** (e.g., FAISS, Chroma, Pinecone, or another supported option).  \n   - Add the document embeddings to the store.  \n   - The vector store supports similarity search by comparing query embeddings to stored vectors.\n\n4. **Create a Retriever**  \n   - Use the vector store’s retriever interface to query for passages most 

In [18]:
response = rag_chain.invoke("How to load and prepare documents for semantic search engine using LangChain?")
response

'To load and prepare documents for a semantic search engine using **LangChain**, you start by using the **Document** abstraction and supporting tools provided in the framework.  \n\n**Step 1: Load Documents**  \nLangChain uses *document loaders* to bring text data into your environment. Each loaded document is represented by a `Document` object with three main attributes:\n- **page_content** – the text of the document or chunk,  \n- **metadata** – a dictionary storing contextual info (e.g., source, page numbers),  \n- **id** – an optional unique identifier.\n\nYou can generate or load documents from various sources such as PDF files using the `pypdf` package:\n```python\nfrom langchain_core.documents import Document\n# Example creation of a document\ndoc = Document(page_content="some text", metadata={"source": "example.pdf"})\n```\n\n**Step 2: Split Text (if needed)**  \nSince large documents can be difficult to handle as a single block of text, LangChain provides **text splitters** to